# Test-benchmark using fake data

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import warnings
warnings.filterwarnings("ignore")

# to remplace with pip install abench and uqmodels
path = '../'
sys.path.insert(1, path)
sys.path.insert(1, path+'src')
import abench as abench
import uqmodels as uqmodels
import abench.store as A_store
import matplotlib.pyplot as plt
import numpy as np
import pickle

# Preprocessing

# Formalisation modèle

In [8]:
#Data_processor for preprocessed data stored in a dict
import abench.store as A_store
from abench.benchmark import dataset_generator_from_array,splitter,TimeSeriesSplit,analyse_data_generator
from src.Benchmark_UQ import Encapsulated_model_UQ,TimeSeries_from_dict
from src.attack_function import ctx_seq_attack_UC_AL, ctx_seq_attack, test_attack
from uqmodels.utils import compute_born

# Type_experiments : Training_inj or Inference_inj
Type_exp = "Inference_inj" # "Inference_inj"

# Load Synthetic data
Folder_data = path + 'data/synthetic_data/'
Name_data = 'synthetic_data_multivariate.p'
Synthetic_dataset = TimeSeries_from_dict(Folder_data+Name_data)
Synthetic_dataset.process()
X,y,context,train,test, X_split = Synthetic_dataset.get_data()
y = y[:,1:2] # Reduce to an Univariate forecast task
X_shape,y_shape = X.shape[-1],y.shape[-1]

# Preliminairies setup for data_generator
X_train, X_test, y_train, y_test, context_train,context_test = Synthetic_dataset.split_train_test()
context = np.concatenate([context,(context[:,0]<20)[:,None]],axis=1)
sk_split = TimeSeriesSplit(n_splits=2,max_train_size=int(len(X)/2),test_size=int(len(X)/4))
objective = None
dataset_generator = dataset_generator_from_array(X,y,context,objective,
                                                 sk_split=sk_split,
                                                 remove_from_train=None,
                                                 repetition=2,
                                                 cv_list_name=['cv_0','cv_1'])

# Build and store data_generator from data
if(Type_exp=="Training_inj"):
    storing = path+'results/Benchmark_synthetic_learning_injection'
    if not os.path.exists(storing):
        dataset_generator = dataset_generator_from_array(X,y,context,objective,
                                                         sk_split=sk_split,
                                                         remove_from_train=None,
                                                         repetition=2,
                                                         cv_list_name=['cv_0','cv_1'])

        remove_from_train_seq_ctx1_bis = ctx_seq_attack(y,context,1,0.95)
        dataset_generator_deg1_bis = dataset_generator_from_array(X,y,context,objective,
                                                              sk_split=sk_split,
                                                              repetition=2,
                                                              remove_from_train=remove_from_train_seq_ctx1_bis,
                                                              cv_list_name=['cv_0_deg1_seq97','cv_1_deg1_seq97'])


        remove_from_train_seq_ctx1_bis = ctx_seq_attack(y,context,1,1)
        dataset_generator_deg2_bis = dataset_generator_from_array(X,y,context,objective,
                                                              sk_split=sk_split,
                                                              repetition=2,
                                                              remove_from_train=remove_from_train_seq_ctx1_bis,
                                                              cv_list_name=['cv_0_deg1_seq100','cv_1_deg1_seq100'])


        dataset_generator = dataset_generator + dataset_generator_deg1_bis + dataset_generator_deg2_bis
        print('Generation of data-generator at '+storing)
        A_store.store_data_generator(storing,dataset_generator)
    else:
        dataset_generator = A_store.get_data_generator(storing)
        print(storing+' already exists')
    
if(Type_exp=="Inference_inj"):
    from src.attack_function import injection_on_data_generator
    storing = path+'results/Benchmark_synthetic_injection_inference'
    if not os.path.exists(storing):
        dataset_generator = dataset_generator_from_array(X,y,context,objective,
                                                     sk_split=sk_split,
                                                     remove_from_train=None,
                                                     repetition=2,
                                                     cv_list_name=['cv_0','cv_1'])
        list_f = [10,11,0,3,6] # Strongest contribution features using Shape
        type_f = ['Num','Num','Num','Num','Num']
        type_attack = [('small',2),('small',5),('strong',2),('strong',5)]
        force = 0
        dataset_generator_with_attack = injection_on_data_generator(dataset_generator,type_attack,
                                                                    type_f,list_f,force)

        print('Generation of data-generator at '+storing)
        store_data_generator(storing,data_generator)
    else:
        print(storing+' already exists')
        dataset_generator = A_store.get_data_generator(storing)

analyse_data_generator(dataset_generator)

../results/Benchmark_synthetic_injection_inference already exists
% train for each sub sample for each set
cv_0  len 60000 %train/test/drop: [10.   3.3] [25.   8.3] [40.  13.3]
cv_1  len 60000 %train/test/drop: [10.   3.3] [25.   8.3] [40.  13.3]
cv_0_bis1  len 60000 %train/test/drop: [10.   3.3] [25.   8.3] [40.  13.3]
cv_1_bis1  len 60000 %train/test/drop: [10.   3.3] [25.   8.3] [40.  13.3]


In [15]:
from uqmodels.common.basic_NN import build_MSE_loss,build_BNN_loss,mlp,default_callbacks

# MLP - BNN
type_var = None
callbacks = default_callbacks(min_delta=0.005,earlystop_patience=60,reducelr_patience=30,reducelr_factor=0.4,reduce_lr_min_lr=5e-06,verbose=0,)
step=1000
model_param={'dim_in':X.shape[-1],'dim_out':y.shape[-1], 'layers_size':[200,300,200],
             'regularizer_W':(0.00003,0.00003),'name':'','dp':0.02,'type_var':type_var,
             'logvar_min':np.log(0.00001)}
            #(0.0001,0.0001)
training_param = {'epochs':[step,step,step],'b_s':[128,32,64],'l_r':[0.001,0.0005,0.0001],'sample_w':None,
    'list_loss':[build_MSE_loss],'metrics':None,
    'param_loss':[1],'callbacks':callbacks}

MLP_parameters = {'rescale':True,
                      'model_initializer':mlp,
                      'model_parameters':model_param,
                      'training_parameters':training_param,
                      'type_var':type_var}

# RF_dUQ
estimator = RandomForestRegressor(ccp_alpha=1e-05, max_depth=20, max_features=0.5,
                                  max_samples=0.99, min_impurity_decrease=1e-05,
                                  min_samples_leaf=2, min_samples_split=4, n_estimators=50,
                                  random_state=None)

RF_dUQ_parameters = {'estimator':estimator,
                     'pretuned':False,
                     'mode':'sigma',
                     'use_biais':True,
                     'rescale':True}

# PNN_MCDP 
type_var = 'MC_Dropout'
callbacks = default_callbacks(min_delta=0.005,earlystop_patience=60,reducelr_patience=30,reducelr_factor=0.4,reduce_lr_min_lr=5e-06,verbose=0,)
step=1000

model_param={'dim_in':X.shape[-1],'dim_out':y.shape[-1], 'layers_size':[200,300,200],
             'regularizer_W':(0.0003,0.0003),'name':'','dp':0.20,'type_var':type_var,'n_ech':8,
             'logvar_min':np.log(0.00001)}
            #(0.0001,0.0001)
training_param = {'epochs':[step,step,step],'b_s':[128,32,64],'l_r':[0.001,0.0005,0.0005],
                  'sample_w':None,
    'list_loss':[build_BNN_loss],'metrics':[build_MSE_loss(2,True),build_BNN_loss(0.9,True)],
    'param_loss':[1.05],'callbacks':callbacks}

PNN_MCDP_parameters = {'rescale':True,
                      'model_initializer':mlp,
                      'model_parameters':model_param,
                      'training_parameters':training_param,
                      'type_var':type_var}

#EDL 
type_var = 'EDL'
from uqmodels.common.basic_NN import build_EDL_loss,build_BNN_loss,mlp
callbacks = default_callbacks(min_delta=0.0005,earlystop_patience=60,reducelr_patience=30,reducelr_factor=0.4,reduce_lr_min_lr=5e-06,verbose=0)

model_param={'dim_in':X.shape[-1],'dim_out':y.shape[-1], 'layers_size':[200,300,200],
             'regularizer_W':(0.0001,0.0001),'name':'','dp':0.02,'type_var':type_var,'logvar_min':np.log(0.00001)}
         #'regularizer_W':(0.0003,0.0003)
training_param = {'epochs':[step,step,step],'b_s':[200,50,100],
                  'l_r':[0.001,0.0005,0.0001],'sample_w':None,'list_loss':[build_EDL_loss],
                  'metrics':[build_MSE_loss(4,True),build_BNN_loss(0.9,True,type_var)],
                  'callbacks':callbacks,'param_loss':[13e-2]}

EDL_parameters = {'rescale':False,
                  'model_initializer':mlp,
                  'model_parameters':model_param,
                  'training_parameters':training_param,
                  'type_var':type_var}

# MLP - BNN
type_var = "Deep_ensemble"
callbacks = default_callbacks(min_delta=0.0005,earlystop_patience=60,reducelr_patience=30,reducelr_factor=0.4,reduce_lr_min_lr=5e-06,verbose=0,)
step=1000

model_param={'dim_in':X.shape[-1],'dim_out':y.shape[-1], 'layers_size':[200,300,200],
             'train_ratio':0.95,'n_ech':5,'k_fold':None,
             'regularizer_W':(0.001,0.001),'name':'','dp':0.02,'type_var':type_var,
             'logvar_min':np.log(0.00001)}
training_param = {'epochs':[step,step],'b_s':[128,64],'l_r':[0.001,0.0005],
                  'sample_w':None,'list_loss':[build_BNN_loss],
                  'metrics':[build_MSE_loss(2,True),build_BNN_loss(0.9,True)],
                  'param_loss':[1],'callbacks':callbacks}

PNN_DE_parameters = {'model_initializer':mlp,
                     'model_parameters':model_param,
                     'training_parameters':training_param,
                     'type_var':type_var}

# Formalisation tache

In [20]:
from sklearn.ensemble import RandomForestRegressor
from uqmodels.common.random_forest_UQ import PredictorRF_UQ_distangle
from uqmodels.common.neural_network_UQ import NN_var
random = np.random.randint(10000000)

# Transmision des modèles / Initialisateurs de modèles
dict_Predictor={}

dict_Predictor['RF_dUQ'] = {'subpart':PredictorRF_UQ_distangle,'parameters':RF_dUQ_parameters}           
dict_Predictor['MLP'] = {'subpart':NN_var,'parameters':MLP_parameters}    
dict_Predictor['PNN_DE'] = {'subpart':NN_var,'parameters':PNN_DE_parameters}    
dict_Predictor['PNN_MCDP'] = {'subpart':NN_var,'parameters':PNN_MCDP_parameters}    
dict_Predictor['EDL'] = {'subpart':NN_var,'parameters':EDL_parameters}

exp_design=[]
# Spécification du plan d'expérience
exp_design.append([{'name':'RF_dUQ_bis','model':'RF_dUQ'},
                   {'name':'MLP_bis','model':'MLP'}
                   {'name':'PNN_DE_bis','model':'PNN_DE'}
                   {'name':'PNN_MCDP_bis','model':'PNN_MCDP'}
                   {'name':'EDL_bis','model':'EDL'}])

# Model wrapper :
from src.Benchmark_UQ import Encapsulated_model_UQ
# Configuration transmis à abench coté modèle
dict_exp={'encapsulated_model': Encapsulated_model_UQ,
          'tuning_scheme' : {'model':(X[train],y[train])},
          'model' : dict_Predictor,
          'exp_design':exp_design}

#print(list_cv_name_all)

In [21]:
#Load Task encapsulator model

# Metrics definition to encapsulate in Generic_metric

    
from src.Benchmark_UQ import rmse,average_coverage,sharpness
from abench.benchmark import Generic_metric

list_ctx_constraint= None
if(True):
    list_ctx_constraint= [(-2,None,0.5)]
list_metrics=[Generic_metric(rmse,'Rmse_all', mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True),
              Generic_metric(average_coverage,"Cov_ALL", mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True,type_var="all"),
              Generic_metric(sharpness,"Aw_E", mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True,type_var="epistemic"),
              Generic_metric(sharpness,"Aw_A", mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True,type_var="aleatoric")]

obj_param = {'alpha':0.05}
tuning_kwarg = {}

#Start from empty dict

from abench.benchmark import benchmark
cv_list = A_store.get_cv_list(storing)
storing = benchmark(storing,None,dict_exp,obj_param,
                    list_metrics,tuning_kwarg=tuning_kwarg,verbose=0,cv_list=cv_list)


n_experiments : 0
PNN_MCDP_bis
load existant data_set
start cv_0
start_fit
1875/1875 [==============================] - 6s 3ms/step
cv_0 len_train 8000
[(['Rmse_all'], [0.394890228183241, 0.4395592284835785]), (['Cov_ALL'], [0.97975, 0.959]), (['Aw_E'], [0.8092865002813542, 0.8151782910371864]), (['Aw_A'], [1.607894199142769, 1.6012442933448214])]
start cv_1
start_fit
1875/1875 [==============================] - 6s 3ms/step
cv_1 len_train 8000
[(['Rmse_all'], [0.4076735671413856, 0.44721640543337443]), (['Cov_ALL'], [0.98175, 0.96275]), (['Aw_E'], [0.8040160226241312, 0.8587424080471446]), (['Aw_A'], [1.642042807513049, 1.6994090063935923])]
start cv_0_bis1
start_fit
1875/1875 [==============================] - 7s 3ms/step
cv_0_bis1 len_train 8000
[(['Rmse_all'], [0.39921690245964503, 0.44487470175820504]), (['Cov_ALL'], [0.98025, 0.96025]), (['Aw_E'], [0.79989643340918, 0.7957533210364065]), (['Aw_A'], [1.6387460773583926, 1.6262914857261095])]
start cv_1_bis1
start_fit
1875/1875 [===

In [3]:
from abench.benchmark import evaluate, Generic_metric
from src.Benchmark_UQ import rmse,average_coverage,sharpness,Gaussian_NLL

storing = path+'results/Benchmark_synthetic_injection_learning'
cv_list = ['cv_0','cv_1','cv_0_bis1','cv_1_bis1']
list_name = ['MLP','RF_dUQ','PNN_MCDP','PNN_DE','EDL']
list_ctx_constraint= None
list_metrics = [Generic_metric(rmse,'Rmse_all', mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True),
                Generic_metric(average_coverage,"Cov_all", mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True),
                Generic_metric(Gaussian_NLL,"NLL", mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True,type_var='all'),
                Generic_metric(sharpness,"sharpness", mask=None,list_ctx_constraint=list_ctx_constraint,reduce=True,type_var='all'),]
dict_perf = evaluate(storing, list_name, list_metrics,verbose=1,cv_list=cv_list)

cv_0 len_train 8000
cv_1 len_train 8000
cv_0_bis1 len_train 8000
cv_1_bis1 len_train 8000
MLP  |time_fit : 237.57 time_pred : 1.38
Rmse_all Train 0.381 ± 0.001 | TEST 0.428 ± 0.002
Cov_all Train 1.0 ± 0.0 | TEST 1.0 ± 0.0
NLL Train 2.247 ± 0.005 | TEST 2.248 ± 0.005
sharpness Train 14.979 ± 0.078 | TEST 14.979 ± 0.078

RF_dUQ  |time_fit : 10.54 time_pred : 15.18
Rmse_all Train 0.241 ± 0.003 | TEST 0.433 ± 0.002
Cov_all Train 1.0 ± 0.0 | TEST 0.967 ± 0.002
NLL Train 0.068 ± 0.009 | TEST 0.438 ± 0.009
sharpness Train 1.547 ± 0.015 | TEST 1.79 ± 0.013

PNN_MCDP  |time_fit : 641.03 time_pred : 16.34
Rmse_all Train 0.395 ± 0.002 | TEST 0.442 ± 0.002
Cov_all Train 0.982 ± 0.001 | TEST 0.963 ± 0.001
NLL Train 0.347 ± 0.003 | TEST 0.466 ± 0.012
sharpness Train 1.821 ± 0.008 | TEST 1.858 ± 0.05

PNN_DE  |time_fit : 2968.25 time_pred : 6.87
Rmse_all Train 0.396 ± 0.002 | TEST 0.428 ± 0.004
Cov_all Train 0.968 ± 0.001 | TEST 0.95 ± 0.001
NLL Train 0.298 ± 0.002 | TEST 0.401 ± 0.016
sharpness Trai